## tobac example: Tracking deep convection based on VIS from geostationary satellite retrievals

This example notebook demonstrates the use of tobac to track isolated deep convective clouds based on radiances within the VIS using channel 2 (red light - 600 nm) of the GOES-16 imaging instrument in 5-min resolution. The study area is loacted within the CONUS extent of the GOES-E for investigating the formation of deep convection over the Carribean, following the [EUREC4A](eurec4a.eu) initiaive. 

The data used in this example is saved on the cloud of the Amazon Web Services, providing an efficient way of processing satellite data without facing the need of downloading the data. 

In [ ]:
# Import libraries
import requests
import netCDF4
import boto3
from botocore import UNSIGNED
from botocore.config import Config
from pyproj import Proj, transform
from pyproj import transformer
#import rioxarray

import xarray
import numpy as np
import pandas as pd
import os
from six.moves import urllib
from glob import glob

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Import tobac itself:
import tobac

In [ ]:
# Disable a few warnings:
import warnings
warnings.filterwarnings('ignore', category=UserWarning, append=True)
warnings.filterwarnings('ignore', category=RuntimeWarning, append=True)
warnings.filterwarnings('ignore', category=FutureWarning, append=True)
warnings.filterwarnings('ignore',category=pd.io.pytables.PerformanceWarning)

In [ ]:
# For acessing data from AWS bucket define bucket specifics
bucket_name = 'noaa-goes16'
product_name = 'ABI-L2-CMIPC'
year = 2020
day_of_year = 45
hour = 14
band = 2

In [ ]:
# Initialize s3 client. 
s3_client = boto3.client('s3', config=Config(signature_version=UNSIGNED))

In [ ]:
# Define Function for generating keys in S3 bucket
def get_s3_keys(bucket, s3_client, prefix = ''):
    """
    Generate the keys in an S3 bucket.

    :param bucket: Name of the S3 bucket.
    :param prefix: Only fetch keys that start with this prefix (optional).
    """
    
    kwargs = {'Bucket': bucket}

    if isinstance(prefix, str):
        kwargs['Prefix'] = prefix

    while True:
        resp = s3_client.list_objects_v2(**kwargs)
        for obj in resp['Contents']:
            key = obj['Key']
            if key.startswith(prefix):
                yield key

        try:
            kwargs['ContinuationToken'] = resp['NextContinuationToken']
        except KeyError:
            break

In [ ]:
keys = get_s3_keys(bucket_name,
                   s3_client,
                   prefix = f'{product_name}/{year}/{day_of_year:03.0f}/{hour:02.0f}/OR_{product_name}-M6C{band:02.0f}'
                  )


keys = [key for key in keys][0:6]

In [ ]:
# Show file names
print(keys)

In [ ]:
# Request data via file names from AWS S3 Server and store data in one xarray data set
for k in range(len(keys)):
    resp = requests.get(f'https://{bucket_name}.s3.amazonaws.com/{keys[k]}')
    file_name = keys[k].split('/')[-1].split('.')[0]
    nc4_ds = netCDF4.Dataset(file_name, memory = resp.content)
    store = xarray.backends.NetCDF4DataStore(nc4_ds)
    if k == 0:
        DS = xarray.open_dataset(store)    
    else:
        DS2 = xarray.open_dataset(store)
        DS = xarray.combine_nested([DS, DS2], concat_dim=["t"], combine_attrs = "override")
print(DS)

In [ ]:
# Check GOES-R grid projection details
DS.variables["goes_imager_projection"]

In [ ]:
# Define projection properties for transforming coordinates
# Satellite height (perspective_point_height)
sat_h = 35786023

# Satellite longitude (longitude_of_projection_origin)
sat_lon = -75.0

# Satellite sweep (sweep_angle_axis)
sat_sweep = "x"

# The projection x and y coordinates equals
# the scanning angle (in radians) multiplied by the satellite height (http://proj4.org/projections/geos.html)
X = DS.variables['x'][:] * sat_h
Y = DS.variables['y'][:] * sat_h

In [ ]:
# Transform GEOS Projection to Lat-/Lon-Format
p = Proj(proj='geos', h=sat_h, lon_0=sat_lon, sweep=sat_sweep)
XX, YY = np.meshgrid(X, Y)
lons, lats = p(XX, YY, inverse=True)
print(len(lons))
Sys.exit()
lats[np.isnan(DS["CMI"]))] = np.nan
print(lats)
lons[np.isinf(DS["CMI"])] = np.nan
print(np.isnan(X))
Sys.exit()
DS.coords['x'] = lons
DS.coords['y'] = lats
print(DS)


In [ ]:
# Crop to area extent of Tropical Atlantic (Carribean) 
min_lon = -65.0
max_lon = -55.0
min_lat = 10.0
max_lat = 15.0

#DS_subset = band.rio.clip_box(minx=min_lon, miny=min_lat, maxx=max_lon, maxy=max_lat)

In [ ]:
#Set up directory to save output and plots:
savedir='Save'
if not os.path.exists(savedir):
    os.makedirs(savedir)
plot_dir="Plot"
if not os.path.exists(plot_dir):
    os.makedirs(plot_dir)

In [ ]:
# Keyword arguments for the feature detection step
parameters_features={}
parameters_features['position_threshold']='weighted_diff'
parameters_features['sigma_threshold']=0.5
parameters_features['min_num']=1
parameters_features['target']='maximum'
parameters_features['threshold']=0.3
parameters_features['dxy']= 500

In [ ]:
# Feature detection and save results to file:
print('starting feature detection')
Features=tobac.themes.tobac_v1.feature_detection_multithreshold(DS["CMI"],**parameters_features)
Features.to_netcdf(os.path.join(savedir,'Features.nc'))
print('feature detection performed and saved')

In [ ]:
# Keyword arguments for the segmentation step:
parameters_segmentation={}
parameters_segmentation['target']='maximum'
parameters_segmentation['method']='watershed'
parameters_segmentation['threshold']=0.3

In [ ]:
# Perform segmentation and save results to files:
Mask_VIS,Features_VIS=tobac.themes.tobac_v1.segmentation(Features,DS["CMI"],dxy,**parameters_segmentation)
print('segmentation VIS performed, start saving results to files')
Mask_VIS.to_netcdf(os.path.join(savedir,'Mask_Segmentation_VIS.nc'))              
Features_VIS.to_netcdf(os.path.join(savedir,'Features_VIS.nc'))
print('segmentation VIS performed and saved')

In [ ]:
# keyword arguments for linking step
parameters_linking={}
parameters_linking['v_max']=20
parameters_linking['stubs']=2
parameters_linking['order']=1
parameters_linking['extrapolate']=1
parameters_linking['memory']=0
parameters_linking['adaptive_stop']=0.2
parameters_linking['adaptive_step']=0.95
parameters_linking['subnetwork_size']=100
parameters_linking['method_linking']= 'predict'

In [ ]:
# Perform linking and save results to file:
Track=tobac.themes.tobac_v1.linking_trackpy(Features,DS["CMI"],dt=dt,dxy=dxy,**parameters_linking)
Track.to_netcdf(os.path.join(savedir,'Track.nc'))